<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/longstaff_schwartz_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch

def generate_asset_paths(S0, sigma, T, r, Nt, Np, Z):
    """
    Generate asset paths for a given set of parameters using the equation provided in the first image.

    Args:
        S0 (float): Initial stock price.
        sigma (float): Volatility.
        T (float): Time to maturity.
        r (float): Risk-free rate.
        Nt (int): Number of time steps.
        Np (int): Number of paths.
        Z (torch.Tensor): Standard normal random variables (size: Np x Nt).

    Returns:
        torch.Tensor: Simulated asset paths (size: Np x Nt).
    """
    dt = torch.tensor(T / Nt)
    t = torch.arange(1, Nt + 1).float() * dt
    drift = (r - 0.5 * sigma ** 2) * t / Nt
    diffusion = sigma * torch.cumsum(Z, dim=1) * torch.sqrt(dt)
    paths = S0 * torch.exp(drift.unsqueeze(0) + diffusion)
    return paths

def longstaff_schwartz(S0, sigma, T, r, Nt, Np, K):
    """
    Implement the Longstaff-Schwartz algorithm for American option pricing.

    Args:
        S0 (float): Initial stock price.
        sigma (float): Volatility.
        T (float): Time to maturity.
        r (float): Risk-free rate.
        Nt (int): Number of time steps.
        Np (int): Number of paths.
        K (float): Strike price.

    Returns:
        float: Estimated option value.
    """
    dt = torch.tensor(T / Nt)
    Z = torch.randn(Np, Nt)  # Standard normal random variables
    paths = generate_asset_paths(S0, sigma, T, r, Nt, Np, Z)

    vp = torch.zeros(Np)
    tp = torch.zeros(Np, dtype=torch.long)

    # Final cash flow at maturity
    vp = torch.maximum(K - paths[:, -1], torch.tensor(0.0))
    tp = torch.full((Np,), Nt, dtype=torch.long)

    # Iterate backward in time
    for t in range(Nt - 1, 0, -1):
        discount_factor = torch.exp(-r * dt)
        vp = vp * discount_factor

        in_the_money = paths[:, t - 1] < K
        exercise_value = K - paths[:, t - 1]

        # Perform regression to find continuation value
        X = paths[in_the_money, t - 1]
        Y = vp[in_the_money]

        if len(X) > 0:
            A = torch.stack([torch.ones_like(X), X, X ** 2], dim=1)
            beta = torch.linalg.lstsq(A, Y).solution
            continuation_value = A @ beta

            # Decide whether to exercise or continue
            exercise_indices = in_the_money.nonzero(as_tuple=True)[0]
            for i, idx in enumerate(exercise_indices):
                if exercise_value[idx] > continuation_value[i]:
                    vp[idx] = exercise_value[idx]
                    tp[idx] = t

    # Compute the option value
    V = torch.mean(vp) * torch.exp(-r * dt * tp.float().mean())
    return V.item()

# Parameters
S0 = 100.0
sigma = 0.25
T = 180/365
r = 0.05
Nt = 1000
Np = 5000
K = 95.0

# Execute the Longstaff-Schwartz algorithm
option_value = longstaff_schwartz(S0, sigma, T, r, Nt, Np, K)
print(f"Estimated Option Value: {option_value:.4f}")

Estimated Option Value: 1.1780


In [1]:
!git clone https://github.com/luphord/longstaff_schwartz.git

Cloning into 'longstaff_schwartz'...
remote: Enumerating objects: 1046, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 1046 (delta 128), reused 221 (delta 114), pack-reused 791 (from 1)
Receiving objects: 100% (1046/1046), 12.07 MiB | 14.58 MiB/s, done.
Resolving deltas: 100% (642/642), done.


In [ ]:
!ls -ltr

total 8
drwxr-xr-x 1 root root 4096 Jan 16 14:29 sample_data
drwxr-xr-x 9 root root 4096 Jan 21 16:54 longstaff_schwartz


In [ ]:
!pip install ./longstaff_schwartz

In [2]:
%cd longstaff_schwartz

/content/longstaff_schwartz


In [4]:
from longstaff_schwartz.algorithm import longstaff_schwartz
from longstaff_schwartz.stochastic_process import GeometricBrownianMotion
import numpy as np


T = 180/365
#Np = 5000, NT = 1000.
# Model parameters
t = np.linspace(0, T, 5000)  # timegrid for simulation
r = 0.05  # riskless rate
sigma = 0.25  # annual volatility of underlying
n = 5000  # number of simulated paths
S0 = 100.0
K  = 95.0

# Simulate the underlying
gbm = GeometricBrownianMotion(mu=r, sigma=sigma)
rnd = np.random.RandomState(1234)
x = gbm.simulate(t, n, rnd)  # x.shape == (t.size, n)

x *= S0

# Payoff (exercise) function
strike = 95.0

def put_payoff(spot):
    return np.maximum(strike - spot, 0.0)

# Discount factor function
def constant_rate_df(t_from, t_to):
    return np.exp(-r * (t_to - t_from))

# Approximation of continuation value
#def fit_quadratic(x, y):
#    return np.polynomial.Polynomial.fit(x, y, 2, rcond=None)

# Approximation of continuation value
def fit_quadratic(x, y):
    if len(x) == 0 or len(y) == 0:
        return lambda x: np.zeros_like(x, dtype=float)
    try:
        # Attempt to fit using the original method
        return np.polynomial.Polynomial.fit(x, y, 2, rcond=None)
    except np.linalg.LinAlgError:
        # If SVD fails, use a more robust method like 'lstsq' directly
        A = np.vstack([x**0, x**1, x**2]).T  # Construct the design matrix
        coeffs, _, _, _ = np.linalg.lstsq(A, y, rcond=None)
        return np.polynomial.Polynomial(coeffs)  # Return a Polynomial object

# Selection of paths to consider for exercise
# (and continuation value approxmation)
def itm(payoff, spot):
    return payoff > 0

# Run valuation of American put option
npv_american = longstaff_schwartz(x, t, constant_rate_df,
                                  fit_quadratic, put_payoff, itm)

# European put option for comparison
npv_european = constant_rate_df(t[0], t[-1]) * put_payoff(x[-1]).mean()

# Check results
#assert np.round(npv_american, 4) == 0.0734
#assert np.round(npv_european, 4) == 0.0626
#assert npv_american > npv_european

print(f"npv_american: {npv_american:.4f}")
print(f"npv_european: {npv_european:.4f}")

npv_american: 3.7927
npv_european: 3.6678


/usr/local/lib/python3.11/dist-packages/numpy/polynomial/polynomial.py:1362: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)
/usr/local/lib/python3.11/dist-packages/numpy/polynomial/polyutils.py:303: RuntimeWarning: divide by zero encountered in scalar divide
  off = (old[1]*new[0] - old[0]*new[1])/oldlen
/usr/local/lib/python3.11/dist-packages/numpy/polynomial/polyutils.py:304: RuntimeWarning: divide by zero encountered in scalar divide
  scl = newlen/oldlen
/usr/local/lib/python3.11/dist-packages/numpy/polynomial/polyutils.py:372: RuntimeWarning: invalid value encountered in add
  return off + scl*x


In [8]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
#import tf_quant_finance as tff

# Parameters
S0 = 100.0  # Initial stock price
K = 95.0    # Strike price
sigma = 0.25  # Volatility
T = 180 / 365  # Time to maturity (in years)
r = 0.05  # Risk-free rate

Np = 100000  # Number of paths
NT = 1000  # Number of time steps
dt = T / NT  # Time step size

# Generate random numbers for paths
Zp = np.random.normal(size=(Np, NT))

# Drift and volatility functions
def drift_fn(_, x):
    return r - sigma**2 * tf.ones_like(x) / 2.0

def vol_fn(_, x):
    return sigma * tf.expand_dims(tf.ones_like(x), axis=-1)

# Function to propagate paths using Euler scheme
def propagate_paths(S0, sigma, T, r, NT, Zp):
    dt = T / NT
    S = np.zeros((Np, NT + 1))
    S[:, 0] = S0
    for t in range(1, NT + 1):
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Zp[:, t-1])
    return S

# Initialize variables
S = propagate_paths(S0, sigma, T, r, NT, Zp)
v = np.zeros(Np)
t_tilde = np.zeros(Np)

# Step 1: Calculate payoff at maturity
for p in range(Np):
    if S[p, NT] < K:
        v[p] = K - S[p, NT]
        t_tilde[p] = NT
    else:
        v[p] = 0

# Step 2: Backward induction
for t in range(NT - 1, 0, -1):
    I = []
    for p in range(Np):
        v[p] *= np.exp(-r * T / NT)
        S[p, t] = S[p, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Zp[p, t-1])
        if S[p, t] < K:
            I.append(p)

    # Regression to find exercise boundary
    if len(I) > 0:
        X = S[I, t]
        Y = v[I]
        b_coeffs = np.polyfit(X, Y, 2)  # Polynomial regression of degree 2

        for p in I:
            # Evaluate the polynomial at the specific point S[p, t]
            b_val = np.polyval(b_coeffs, S[p, t])
            if S[p, t] < b_val:
                v[p] = K - S[p, t]
                t_tilde[p] = t

# Step 3: Calculate option value
V = np.sum(v) * np.exp(-r * T / NT) / Np
print(f"Option Value: {V:.4f}")

<ipython-input-8-1b853ccf506c>:62: RankWarning: Polyfit may be poorly conditioned
  b_coeffs = np.polyfit(X, Y, 2)  # Polynomial regression of degree 2


Option Value: 3.6962


In [10]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
#import tf_quant_finance as tff
import warnings

# Parameters
S0 = 100.0  # Initial stock price
K = 95.0    # Strike price
sigma = 0.25  # Volatility
T = 180 / 365  # Time to maturity (in years)
r = 0.05  # Risk-free rate

Np = 50000  # Number of paths
NT = 1000  # Number of time steps
dt = T / NT  # Time step size

# Generate random numbers for paths
Zp = np.random.normal(size=(Np, NT))

# Drift and volatility functions
def drift_fn(_, x):
    return r - sigma**2 * tf.ones_like(x) / 2.0

def vol_fn(_, x):
    return sigma * tf.expand_dims(tf.ones_like(x), axis=-1)

# Function to propagate paths using Euler scheme
def propagate_paths(S0, sigma, T, r, NT, Zp):
    dt = T / NT
    S = np.zeros((Np, NT + 1))
    S[:, 0] = S0
    for t in range(1, NT + 1):
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Zp[:, t-1])
    return S

# Suppress RankWarning
warnings.filterwarnings('ignore', category=np.RankWarning)

# Initialize variables
S = propagate_paths(S0, sigma, T, r, NT, Zp)
v = np.zeros(Np)
t_tilde = np.zeros(Np)

# Step 1: Calculate payoff at maturity
for p in range(Np):
    if S[p, NT] < K:
        v[p] = K - S[p, NT]
        t_tilde[p] = NT
    else:
        v[p] = 0

# Step 2: Backward induction
for t in range(NT - 1, 0, -1):
    I = []
    for p in range(Np):
        v[p] *= np.exp(-r * T / NT)
        S[p, t] = S[p, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Zp[p, t-1])
        if S[p, t] < K:
            I.append(p)

    # Regression to find exercise boundary
    if len(I) > 0:
        X = S[I, t]
        Y = v[I]

        # Normalize data for better numerical stability
        X_mean = np.mean(X)
        X_std = np.std(X)

        # Check for zero variance and handle it
        if X_std == 0:
            # If variance is zero, all X values are the same
            # Choose a simple continuation value, e.g., average of Y
            b_val = np.mean(Y) if len(Y) > 0 else 0
        else:
            X_normalized = (X - X_mean) / X_std
            b_coeffs = np.polyfit(X_normalized, Y, 2)  # Polynomial regression of degree 2
            S_normalized = (S[p, t] - X_mean) / X_std  # Normalize S[p, t] using the same parameters
            b_val = np.polyval(b_coeffs, S_normalized)


        for p in I:
            # Evaluate the polynomial at the specific point S[p, t]

            if S[p, t] < b_val:
                v[p] = K - S[p, t]
                t_tilde[p] = t

# Step 3: Calculate option value
V = np.sum(v) * np.exp(-r * T / NT) / Np
print(f"Option Value: {V:.4f}")

Option Value: 3.6759


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
#import tf_quant_finance as tff
import warnings

# Parameters
S0 = 100.0  # Initial stock price
K = 95.0    # Strike price
sigma = 0.25  # Volatility
T = 180 / 365  # Time to maturity (in years)
r = 0.05  # Risk-free rate

Np = 50000  # Number of paths
NT = 1000  # Number of time steps
dt = T / NT  # Time step size

# Generate random numbers for paths
Zp = np.random.normal(size=(Np, NT))

# Drift and volatility functions
def drift_fn(_, x):
    return r - sigma**2 * tf.ones_like(x) / 2.0

def vol_fn(_, x):
    return sigma * tf.expand_dims(tf.ones_like(x), axis=-1)

# Function to propagate paths using Euler scheme
def propagate_paths(S0, sigma, T, r, NT, Zp):
    dt = T / NT
    S = np.zeros((Np, NT + 1))
    S[:, 0] = S0
    for t in range(1, NT + 1):
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Zp[:, t-1])
    return S

# Suppress RankWarning
warnings.filterwarnings('ignore', category=np.RankWarning)

# Initialize variables
S = propagate_paths(S0, sigma, T, r, NT, Zp)
v = np.zeros(Np)
t_tilde = np.zeros(Np)

# Step 1: Calculate payoff at maturity
for p in range(Np):
    if S[p, NT] < K:
        v[p] = K - S[p, NT]
        t_tilde[p] = NT
    else:
        v[p] = 0

# Step 2: Backward induction
for t in range(NT - 1, 0, -1):
    I = []
    for p in range(Np):
        v[p] *= np.exp(-r * T / NT)
        S[p, t] = S[p, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Zp[p, t-1])
        if S[p, t] < K:
            I.append(p)

    # Regression to find exercise boundary
    if len(I) > 0:
        X = S[I, t]
        Y = v[I]

        # Normalize data for better numerical stability
        X_mean = np.mean(X)
        X_std = np.std(X)

        # Check for zero variance and handle it
        if X_std == 0:
            # If variance is zero, all X values are the same
            # Choose a simple continuation value, e.g., average of Y
            b_val = np.mean(Y) if len(Y) > 0 else 0
        else:
            X_normalized = (X - X_mean) / X_std
            b_coeffs = np.polyfit(X_normalized, Y, 2)  # Polynomial regression of degree 2
            S_normalized = (S[p, t] - X_mean) / X_std  # Normalize S[p, t] using the same parameters
            b_val = np.polyval(b_coeffs, S_normalized)


        for p in I:
            # Evaluate the polynomial at the specific point S[p, t]

            if S[p, t] < b_val:
                v[p] = K - S[p, t]
                t_tilde[p] = t

# Step 3: Calculate option value
V = np.sum(v) * np.exp(-r * T / NT) / Np
print(f"Option Value: {V:.4f}")

# Automatic differentiation to calculate sensitivities
S0_tf = tf.Variable(S0, dtype=tf.float32)
sigma_tf = tf.Variable(sigma, dtype=tf.float32)
r_tf = tf.Variable(r, dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    S_tf = propagate_paths(S0_tf.numpy(), sigma_tf.numpy(), T, r_tf.numpy(), NT, Zp)
    v_tf = np.zeros(Np)
    t_tilde_tf = np.zeros(Np)

    # Step 1: Calculate payoff at maturity
    for p in range(Np):
        if S_tf[p, NT] < K:
            v_tf[p] = K - S_tf[p, NT]
            t_tilde_tf[p] = NT
        else:
            v_tf[p] = 0

    # Step 2: Backward induction
    for t in range(NT - 1, 0, -1):
        I = []
        for p in range(Np):
            v_tf[p] *= np.exp(-r_tf.numpy() * T / NT)
            S_tf[p, t] = S_tf[p, t-1] * np.exp((r_tf.numpy() - 0.5 * sigma_tf.numpy()**2) * dt + sigma_tf.numpy() * np.sqrt(dt) * Zp[p, t-1])
            if S_tf[p, t] < K:
                I.append(p)

        # Regression to find exercise boundary
        if len(I) > 0:
            X = S_tf[I, t]
            Y = v_tf[I]

            # Normalize data for better numerical stability
            X_mean = np.mean(X)
            X_std = np.std(X)

            # Check for zero variance and handle it
            if X_std == 0:
                b_val = np.mean(Y) if len(Y) > 0 else 0
            else:
                X_normalized = (X - X_mean) / X_std
                b_coeffs = np.polyfit(X_normalized, Y, 2)  # Polynomial regression of degree 2
                S_normalized = (S_tf[p, t] - X_mean) / X_std  # Normalize S[p, t] using the same parameters
                b_val = np.polyval(b_coeffs, S_normalized)

            for p in I:
                if S_tf[p, t] < b_val:
                    v_tf[p] = K - S_tf[p, t]
                    t_tilde_tf[p] = t

    # Step 3: Calculate option value
    V_tf = np.sum(v_tf) * np.exp(-r_tf.numpy() * T / NT) / Np

# Calculate sensitivities
delta = tape.gradient(V_tf, S0_tf)
vega = tape.gradient(V_tf, sigma_tf)
rho = tape.gradient(V_tf, r_tf)

print(f"Delta: {delta.numpy():.4f}")
print(f"Vega: {vega.numpy():.4f}")
print(f"Rho: {rho.numpy():.4f}")

Option Value: 3.6228


In [14]:
import torch
import warnings

# Parameters
S0 = 100.0  # Initial stock price
K = 95.0    # Strike price
sigma = 0.25  # Volatility
T = 180 / 365  # Time to maturity (in years)
r = 0.05  # Risk-free rate

Np = 50000  # Number of paths
NT = 1000  # Number of time steps
dt = torch.tensor(T / NT)  # Time step size as a tensor

# Generate random numbers for paths
Zp = torch.randn(Np, NT)

# Function to propagate paths using Euler scheme
def propagate_paths(S0, sigma, T, r, NT, Zp):
    dt = torch.tensor(T / NT)
    S = torch.zeros((Np, NT + 1))
    S[:, 0] = S0
    for t in range(1, NT + 1):
        S[:, t] = S[:, t-1] * torch.exp((r - 0.5 * sigma**2) * dt + sigma * torch.sqrt(dt) * Zp[:, t-1])
    return S

# Suppress warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Initialize variables
S = propagate_paths(S0, sigma, T, r, NT, Zp)
v = torch.zeros(Np)
t_tilde = torch.zeros(Np, dtype=torch.long)

# Step 1: Calculate payoff at maturity
for p in range(Np):
    if S[p, NT] < K:
        v[p] = K - S[p, NT]
        t_tilde[p] = NT
    else:
        v[p] = 0

# Step 2: Backward induction
for t in range(NT - 1, 0, -1):
    I = []
    for p in range(Np):
        v[p] *= torch.exp(-r * dt)
        S[p, t] = S[p, t-1] * torch.exp((r - 0.5 * sigma**2) * dt + sigma * torch.sqrt(dt) * Zp[p, t-1])
        if S[p, t] < K:
            I.append(p)

    # Regression to find exercise boundary
    if len(I) > 0:
        X = S[I, t]
        Y = v[I]

        # Normalize data for better numerical stability
        X_mean = torch.mean(X)
        X_std = torch.std(X)

        if X_std == 0:
            # Handle zero variance case
            b_val = torch.mean(Y) if len(Y) > 0 else 0
        else:
            X_normalized = (X - X_mean) / X_std
            b_coeffs = torch.polyfit(X_normalized, Y, 2)  # Polynomial regression of degree 2
            S_normalized = (S[p, t] - X_mean) / X_std
            b_val = torch.polyval(b_coeffs, S_normalized)

        for p in I:
            if S[p, t] < b_val:
                v[p] = K - S[p, t]
                t_tilde[p] = t

# Step 3: Calculate option value
V = torch.sum(v) * torch.exp(-r * dt) / Np
print(f"Option Value: {V:.4f}")

TypeError: sqrt(): argument 'input' (position 1) must be Tensor, not float